In [1]:
#include "xtensor/xtensor.hpp"
#include "xtensor/xarray.hpp"
#include "xtensor/xio.hpp"
#include "include/xquaternion.hpp"

using namespace std;

# Basics

In [2]:
xt::ones<xt::quaternion<double>>({3})

"Q: 1, 0, 0, 0"
"Q: 1, 0, 0, 0"
"Q: 1, 0, 0, 0"


In [3]:
xt::quaternion<double> q = {1, 2, 3, 4};
cout << q;

Q: 1, 2, 3, 4


In [4]:
xt::conj(q)

"Q: 1, -2, -3, -4"


In [5]:
q = 2;
cout << q;

Q: 2, 0, 0, 0


In [6]:
xt::xtensor<xt::quaternion<double>, 1> qt = {{1, 2, 3, 4}, {5, 6, 7, 8}};

In [7]:
qt*xt::conj(qt)

"Q: 30, 0, 0, 0"
"Q: 174, 0, 0, 0"


In [8]:
qt - q

"Q: -1, 2, 3, 4"
"Q: 3, 6, 7, 8"


# Views

In [9]:
xt::offset_view(qt, y) =  xt::zeros<double>({2});
qt

"Q: 1, 2, 0, 4"
"Q: 5, 6, 0, 8"


In [10]:
xt::xarray<double> qr = {xt::raw_view(qt)};
qr

1.,2.,0.,4.
5.,6.,0.,8.


In [11]:
xt::xarray<xt::quaternion<double>>{xt::quat_view(qr)}

"Q: 1, 2, 0, 4"
"Q: 5, 6, 0, 8"


# Coordinate Transform

In [12]:
#include "xtensor/xrandom.hpp"
#include "xtensor/xpad.hpp"
#include "xtensor/xstrided_view.hpp"

In [13]:
xt::xarray<double> p = xt::random::rand<double>({2, 2, 3});
xt::quaternion<double> rotation = {1, 2, 3, 4};
xt::xtensor<double, 1> offset = {1, 2, 3};
p

0.135477,0.835009,0.968868
0.221034,0.308167,0.547221
0.188382,0.992881,0.996461
0.967695,0.725839,0.98111


In [14]:
template <class E1, class E2, class E3>
auto rel2abs(E1&& p, E2&& offset, E3&& rotation) {
    auto shape = p.shape();
    vector<size_t> vshape(shape.begin(), shape.end());
    vector<vector<size_t>> padw(vshape.size(), {0, 0});
    padw.back()[0] = 1;
    
    // array of cartesian vectors to array quaternions with w = 0
    xt::xarray<xt::quaternion<double>> p_q = xt::quat_view(
        xt::pad(xt::eval(p + offset), padw));
    
    // rotate array of quaternions and strip w dimension
    xt::xarray<double> p_trans = xt::strided_view(
        xt::raw_view(xt::eval(rotation*p_q*xt::conj(rotation))), 
        {xt::ellipsis(), xt::range(1, 4)});
        
    return p_trans;
}

In [15]:
rel2abs(p, offset, rotation)

75.945585,73.73681,106.610482
62.85084,72.283422,91.0279
76.126035,73.768053,111.670342
59.13387,91.717703,111.924879
